In [1]:
import time
from collections import deque
import pandas as pd

In [2]:
class Scraper:
    # required
    def __init__(self, driver, home_url):
        self.driver = driver
        self.home_url = home_url

    # you can do these three with your group
    def easter_egg(self):
        self.driver.get(self.home_url)
        txt = str()
        content = self.driver.find_elements_by_tag_name("span")
        for word in content:
            txt += word.text
        return txt

    def dfs_pass(self):
        self.password = str()
        self.btn = "DFS"
        self.children = []
        
        self.driver.get(self.home_url)
        link = self.driver.find_element_by_tag_name("a")
        url = link.get_attribute("href")
        self.added = {url,}
        
        entrance_btn = self.driver.find_element_by_id("maze-entrance")
        entrance_btn.click()
        
        self.find(url)
        
        return self.password
        
    def find(self,url): # DFS: depth first search
        # click DFS button and get letter
        self.driver.get(url)
        btn = self.driver.find_element_by_id(self.btn)
        btn.click()
        letter = self.driver.find_element_by_id(self.btn)
        self.password += letter.text
        
        # get hyperlinks
        links = self.driver.find_elements_by_tag_name("a")
        hyperlinks = [link.get_attribute("href") for link in links]
        
        for hyperlink in hyperlinks:
            if not hyperlink in self.added:
                self.added.add(hyperlink)
                self.find(hyperlink)
    
    def visit_page(self, url):
        self.driver.get(url)
        links = self.driver.find_elements_by_tag_name("a")
        return [link.get_attribute("href") for link in links]                

    def visit_next(self):
        # 1. do the first thing on TODO list
        url = self.todo.popleft()
        children_urls = self.visit_page(url)

        # 2. add new urls to the end of TODO list
        for child_url in children_urls:
            if not child_url in self.added:
                self.todo.append(child_url)
                self.added.add(child_url)
                
        # click the BFS button, get letter
        btn = self.driver.find_element_by_id(self.btn)
        btn.click()
        letter = self.driver.find_element_by_id(self.btn)
        
        return letter.text

    def bfs_pass(self):
        self.password = str()
        self.btn = "BFS"
        
        self.driver.get(self.home_url)
        link = self.driver.find_element_by_tag_name("a")
        url = link.get_attribute("href")
        
        self.todo = deque([url])
        self.added = {url,}
        
        entrance_btn = self.driver.find_element_by_id("maze-entrance")
        entrance_btn.click()    
        
        while len(self.todo) > 0:
            self.password += self.visit_next()
        return self.password
        
    # write the code for this one individually
    def protected_df(self, password):
        self.driver.get(self.home_url)
        pwd = self.driver.find_element_by_id("password-input")
        pwd.clear()
        pwd.send_keys(password)
        btn = self.driver.find_element_by_id("attempt-button")
        btn.click()
                         
        x = -1
        while len(self.driver.find_elements_by_tag_name("tr")) > x:
            x = len(self.driver.find_elements_by_tag_name("tr"))
            time.sleep(1)
            try:
                btn = self.driver.find_element_by_id("more-locations-button")
                btn.click()
            except:
                continue
                
        return pd.read_html(self.driver.page_source)[0]

In [3]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# from scrape import Scraper
os.system("pkill chrome")

url = "http://23.251.147.53:5000/" # TODO: enter your VM's IP
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)
s = Scraper(driver, url)

In [5]:
m=s.protected_df(bpass)

In [10]:
m[0]

,ID,Latitude,Longitude,Access Code,Description
0,1,43.089034,-89.416128,983kbsdfk1,Picnic Point in Madison
1,2,38.105507,126.910613,37461983fd,Silver Beach in Hawaii
2,3,65.044901,-16.712836,jnjsd238yf,Shore of a Volcanic Lake in Iceland
3,4,48.860945,2.335773,7733hhfsdf,The Louvre in Paris
4,5,37.434183,-122.321990,ksjfn21213,Redwood forest in San Francisco
5,6,51.180315,-1.829659,348219389f,Stonehenge in the UK
6,7,27.987586,86.925002,njsg1hywov,Mt. Everest in Nepal
7,8,43.070010,-89.409450,8wbd1vy29a,Quick Trip on Monroe Street


In [4]:
dpass = s.dfs_pass()
print("\nDFS Password", dpass)

bpass = s.bfs_pass()
print("\nBFS Password", bpass)

print("\nDFS Locations")
print(s.protected_df(dpass))

print("\nBFS Locations")
print(s.protected_df(bpass))



DFS Password BleuSus

BFS Password XÆ_A-12

DFS Locations
[   ID   Latitude   Longitude Access Code                     Description
0   9  38.624912  -90.186127  83ucbsobyq        Gateway Arch in St.Louis
1  10 -13.162773  -72.545824  sjnvjn27ap              Machupiccu in Peru
2  11  43.036319  -89.415369  djhsfdhf31              Culvers in Madison
3  12  39.917529  116.392320  qazwxs23df         Forbidden City in China
4  13  29.979174   31.133935  ewiufb35sf  Great Pyramid of Giza in Egypt
5  14  40.774200  -73.970357  37jdvnag4s        Central Park in New York
6  15 -78.464419  106.826764  ujsdfi274x    Vostok Station in Antarctica
7  16  11.425367  142.811867  8hfnskjdnf          Mariana Trench by Guam]

BFS Locations
[   ID   Latitude   Longitude Access Code                          Description
0   1  43.089034  -89.416128  983kbsdfk1              Picnic Point in Madison
1   2  38.105507  126.910613  37461983fd               Silver Beach in Hawaii
2   3  65.044901  -16.712836  jn